In [1]:
import pandas as pd
import numpy as np
from pprint import pprint
from datetime import datetime

import gspread
from df2gspread import df2gspread as d2g
from googleapiclient import discovery
from google.oauth2 import service_account
from oauth2client.service_account import ServiceAccountCredentials
from googleapiclient.discovery import build

# ga_keys = 'creds.json'
ga_keys = '/Users/proninsergei/pronins/BI/proninsgs-9e4ce4884ed2.json'


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [4]:
pd.set_option('display.max_columns', None)


file_path = '/Users/proninsergei/Downloads/MarketplaceAnalytics/WB/2do/WB_Товары_для_взрослых_Секс_игрушки_Товары_22052023_20062023_21062023.csv'
SexToys = pd.read_csv(file_path,delimiter = ';', header = 0)
SexToys


/var/folders/_2/40qf6ytd5b7cngy29rh2bttr0000gn/T/ipykernel_1949/434765795.py:5: DtypeWarning: Columns (12,16,17,18,19,24,36) have mixed types. Specify dtype option on import or set low_memory=False.
  SexToys = pd.read_csv(file_path,delimiter = ';', header = 0)


,SKU,Name,Category,Brand,Seller,Color,Balance,Balance FBS,Comments,Final price,Max price,Min price,Average price,Median price,Sales,Revenue,Revenue potential,Revenue average,Lost profit,Lost profit percent,URL,Thumb,Days in stock,Days with sales,Average if in stock,Rating,Basic Sale,Basic Sale Price,Promo Sale,Base price,SPP,SPP Price,FBS,Subject ID,Category Position,Categories Last Count,Sales Per Day Average,Country,Gender,First Date,First Comment Date,Pics Count,Has 3D,Hax Video,Comments Valuation,Card Rating,Subject,Purchase,Purchase After Return,22.05.2023 Sales,23.05.2023 Sales,24.05.2023 Sales,25.05.2023 Sales,26.05.2023 Sales,27.05.2023 Sales,28.05.2023 Sales,29.05.2023 Sales,30.05.2023 Sales,31.05.2023 Sales,01.06.2023 Sales,02.06.2023 Sales,03.06.2023 Sales,04.06.2023 Sales,05.06.2023 Sales,06.06.2023 Sales,07.06.2023 Sales,08.06.2023 Sales,09.06.2023 Sales,10.06.2023 Sales,11.06.2023 Sales,12.06.2023 Sales,13.06.2023 Sales,14.06.2023 Sales,15.06.2023 Sales,16.06.2023 Sales,17.06.2023 Sales,18.06.2023 Sales,19.06.2023 Sales,20.06.2023 Sales,22.05.2023 Revenue,23.05.2023 Revenue,24.05.2023 Revenue,25.05.2023 Revenue,26.05.2023 Revenue,27.05.2023 Revenue,28.05.2023 Revenue,29.05.2023 Revenue,30.05.2023 Revenue,31.05.2023 Revenue,01.06.2023 Revenue,02.06.2023 Revenue,03.06.2023 Revenue,04.06.2023 Revenue,05.06.2023 Revenue,06.06.2023 Revenue,07.06.2023 Revenue,08.06.2023 Revenue,09.06.2023 Revenue,10.06.2023 Revenue,11.06.2023 Revenue,12.06.2023 Revenue,13.06.2023 Revenue,14.06.2023 Revenue,15.06.2023 Revenue,16.06.2023 Revenue,17.06.2023 Revenue,18.06.2023 Revenue,19.06.2023 Revenue,20.06.2023 Revenue,22.05.2023 Stocks,23.05.2023 Stocks,24.05.2023 Stocks,25.05.2023 Stocks,26.05.2023 Stocks,27.05.2023 Stocks,28.05.2023 Stocks,29.05.2023 Stocks,30.05.2023 Stocks,31.05.2023 Stocks,01.06.2023 Stocks,02.06.2023 Stocks,03.06.2023 Stocks,04.06.2023 Stocks,05.06.2023 Stocks,06.06.2023 Stocks,07.06.2023 Stocks,08.06.2023 Stocks,09.06.2023 Stocks,10.06.2023 Stocks,11.06.2023 Stocks,12.06.2023 Stocks,13.06.2023 Stocks,14.06.2023 Stocks,15.06.2023 Stocks,16.06.2023 Stocks,17.06.2023 Stocks,18.06.2023 Stocks,19.06.2023 Stocks,20.06.2023 Stocks
0,98732332,Вибратор стимулятор реалистичный с подогревом,Товары для взрослых/Секс игрушки/Вибраторы и с...,DopingLove,ИП Зинченко Д Н,телесный,2205,0,1602,3158,3164,3120,"3155,1631",3158.0,1202,3792506,3792506,"126416,8667",0,0,https://www.wildberries.ru/catalog/98732332/de...,https://basket-05.wb.ru/vol987/part98732/98732...,30,30,"40,0667",5,95,3158,0,63171,20,2526,0,5067,1,3,"40,0667",Китай,NaN,2022-09-16,2022-09-19,18,0,1,"4,59",80,Товары для взрослых / Вибраторы,91,91,25,26,36,35,29,37,43,39,36,36,50,49,44,45,33,43,30,38,40,44,40,41,41,58,53,48,40,43,37,43,79100,82264,113904,110740,90480,115440,134160,123162,113688,113688,157900,154742,138952,142110,104214,135794,94740,120004,126320,138952,126320,129478,129478,183164,167374,151584,126320,135794,116846,135794,1474,1450,1417,1642,2088,2082,2068,2031,1994,2137,2366,2317,2276,2232,2793,2769,2763,2728,2688,2643,2602,2562,2525,2467,2412,2360,2324,2282,2246,2205
1,15028718,"Вибратор вакуумный Пингвин женский, игрушка дл...",Товары для взрослых/Секс игрушки/Вибраторы и с...,Satisfyer,ООО Онмедиа,черный,1396,0,2037,3499,3499,2973,"3480,7772",3499.0,1068,3717470,3717470,"123915,6667",0,0,https://www.wildberries.ru/catalog/15028718/de...,https://basket-02.wb.ru/vol150/part15028/15028...,30,30,"35,6",5,80,3499,0,17495,20,2799,0,5576,1,3,"35,6",NaN,NaN,2020-10-07,2020-11-01,13,0,1,"4,59",83,Товары для взрослых / Вакуумно-волновые стимул...,92,92,31,24,32,28,29,33,17,19,28,37,27,29,33,27,40,27,29,26,23,30,33,45,50,54,59,40,46,55,63,54,108469,83976,111968,97972,101471,115467,59483,66481,97972,110001,94473,101471,115467,94473,139960,94473,101471,90974,80477,104970,115467,157455,174950,188946,206441,139960,160954,192445,220437,188946,1755,1730,1698,1670,1643,1759,1743,1776,1749,1710,1685,1660,1629,1603,1773,1746,1719,1722,1817,1846,1812,1767,1718,1662,16

In [6]:
SexToys_filteres = SexToys.query('Sales >30').sort_values(['Sales'])
SexToys_filteres_df = SexToys_filteres[['Name','Sales','Revenue', 'Average price','Median price',
               'Revenue potential','Lost profit','Category Position',
               'Sales Per Day Average','First Date','Seller','Thumb','Comments',
                'URL','Rating','Comments Valuation','Card Rating','Purchase']]
SexToys_filteres_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2139 entries, 1289 to 6
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Name                   2139 non-null   object 
 1   Sales                  2139 non-null   int64  
 2   Revenue                2139 non-null   int64  
 3   Average price          2139 non-null   object 
 4   Median price           2139 non-null   float64
 5   Revenue potential      2139 non-null   object 
 6   Lost profit            2139 non-null   object 
 7   Category Position      2139 non-null   int64  
 8   Sales Per Day Average  2139 non-null   object 
 9   First Date             2139 non-null   object 
 10  Seller                 2139 non-null   object 
 11  Thumb                  2139 non-null   object 
 12  Comments               2139 non-null   int64  
 13  URL                    2139 non-null   object 
 14  Rating                 2139 non-null   int64  
 15  Comm

In [11]:
import pandas as pd

def classify_SexToy_type(description):
    SexToy_types = []
    

        
    if len(SexToy_types) == 0:
        SexToy_types.append('Unknown')
    return ', '.join(SexToy_types)

# # Загрузка данных из CSV файла
# df = bags_filteres_df

# Применение функции classify_bag_type ко всем строкам в колонке 'description.lower()'
SexToys_filteres_df['SexToy_type'] = SexToys_filteres_df['Name'].apply(classify_SexToy_type)

# Вывод результата
SexToys_filteres_df.head(5)


/var/folders/_2/40qf6ytd5b7cngy29rh2bttr0000gn/T/ipykernel_1949/916446400.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SexToys_filteres_df['SexToy_type'] = SexToys_filteres_df['Name'].apply(classify_SexToy_type)


,Name,Sales,Revenue,Average price,Median price,Revenue potential,Lost profit,Category Position,Sales Per Day Average,First Date,Seller,Thumb,Comments,URL,Rating,Comments Valuation,Card Rating,Purchase,SexToy_type
1289,Вибратор женский для клитора вакуумный с вибра...,31,57145,"1843,3871",1943.0,57145,0,780,"1,0333",2023-03-11,ИП Хисамов И Ш,https://basket-10.wb.ru/vol1497/part149790/149...,21,https://www.wildberries.ru/catalog/149790280/d...,5,5,80,91,Unknown
2708,"Вагинальные шарики, тренажер кегеля",31,21907,"706,6774",697.0,21907,0,61,"1,0333",2021-06-12,ООО АМА Дистрибьюшн,https://basket-02.wb.ru/vol286/part28678/28678...,45,https://www.wildberries.ru/catalog/28678727/de...,5,5,80,92,Unknown
3045,вагинальные шарики тренажер Кегеля вумбилдинг 18+,31,18280,"589,6774",575.0,18280,0,4296,"1,0333",2023-02-10,ИП Валешина,https://basket-10.wb.ru/vol1458/part145862/145...,36,https://www.wildberries.ru/catalog/145862388/d...,5,"4,9",66,92,Unknown
3066,Вибратор,31,18065,"582,7419",584.0,"18687,931","622,931",-1,"1,0333",2023-04-07,Stanetplokhoy,https://basket-10.wb.ru/vol1526/part152672/152...,7,https://www.wildberries.ru/catalog/152672349/d...,4,"3,9",50,91,Unknown
2673,Анальный крюк бдсм,31,22382,722,722.0,22382,0,399,"1,0333",2023-03-23,ИП Ярмоленко Д Г,https://basket-10.wb.ru/vol1520/part152040/152...,7,https://www.wildberries.ru/catalog/152040583/d...,5,5,57,93,Unknown


In [26]:
# def replace_values(bag_type):
#     if 'Man' in bag_type and 'Woman' in bag_type:
#         return bag_type.replace('Man', '').replace('Woman', '') + 'MENWOMEN'
#     return bag_type

# bags_filteres_df['bag_type'] = bags_filteres_df['bag_type'].apply(replace_values)


/var/folders/_2/40qf6ytd5b7cngy29rh2bttr0000gn/T/ipykernel_86179/1307438448.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bags_filteres_df['bag_type'] = bags_filteres_df['bag_type'].apply(replace_values)


In [13]:
SexToys_filteres_df['First Date'] = SexToys_filteres_df['First Date'].astype(str)
SexToys_filteres_df['SexToy_type'] = SexToys_filteres_df['SexToy_type'].astype(str)

# SexToys_filteres_df['Color'].replace('', 'none', inplace=True)
SexToys_filteres_df['URL'] = SexToys_filteres_df['URL'].astype(str)
SexToys_filteres_df['Thumb'] = SexToys_filteres_df['Thumb'].astype(str)
SexToys_filteres_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 2139 entries, 1289 to 6
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Name                   2139 non-null   object 
 1   Sales                  2139 non-null   int64  
 2   Revenue                2139 non-null   int64  
 3   Average price          2139 non-null   object 
 4   Median price           2139 non-null   float64
 5   Revenue potential      2139 non-null   object 
 6   Lost profit            2139 non-null   object 
 7   Category Position      2139 non-null   int64  
 8   Sales Per Day Average  2139 non-null   object 
 9   First Date             2139 non-null   object 
 10  Seller                 2139 non-null   object 
 11  Thumb                  2139 non-null   object 
 12  Comments               2139 non-null   int64  
 13  URL                    2139 non-null   object 
 14  Rating                 2139 non-null   int64  
 15  Comm

/var/folders/_2/40qf6ytd5b7cngy29rh2bttr0000gn/T/ipykernel_1949/3158929009.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SexToys_filteres_df['First Date'] = SexToys_filteres_df['First Date'].astype(str)
/var/folders/_2/40qf6ytd5b7cngy29rh2bttr0000gn/T/ipykernel_1949/3158929009.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SexToys_filteres_df['SexToy_type'] = SexToys_filteres_df['SexToy_type'].astype(str)
/var/folders/_2/40qf6ytd5b7cngy29rh2bttr0000gn/T/ipykernel_1949/3158929009.py:5: SettingWith

In [28]:
# import gspread
# from google.auth import credentials
# from google.auth.transport.requests import Request
# from google.oauth2 import service_account

# # Путь к файлу с учетными данными сервисного аккаунта
# # service_account_file = 'path/to/service_account.json'

# # Функция для аутентификации и получения экземпляра клиента gspread
# def authenticate_with_gspread():
#     # Загрузка учетных данных сервисного аккаунта
#     credentials = service_account.Credentials.from_service_account_file(ga_keys, scopes=['https://www.googleapis.com/auth/spreadsheets'])
    
#     # Проверка и обновление учетных данных, если требуется
#     if credentials.expired and credentials.refresh_token:
#         credentials.refresh(Request())
    
#     # Создание экземпляра клиента gspread
#     client = gspread.authorize(credentials)
    
#     return client

# # Пример использования
# def main():
#     # Аутентификация и получение клиента gspread
#     client = authenticate_with_gspread()
    
#     # Открытие Google Sheets по URL или имени
#     # spreadsheet = client.open_by_url('https://docs.google.com/spreadsheets/d/your_sheet_id/edit')
#     spreadsheet = client.open('WB_analytics')
    
#     # Получение рабочего листа
#     sheet = spreadsheet.sheet1
    
#     # Чтение данных из ячеек
#     cell_value = sheet.cell(1, 1).value
#     print(f"Cell value: {cell_value}")
    
#     # Запись данных в ячейку
#     sheet.update_cell(1, 2, "New Value")
#     print("Cell updated successfully")
    
# if __name__ == '__main__':
#     main()


APIError: {'code': 403, 'message': 'Request had insufficient authentication scopes.', 'errors': [{'message': 'Insufficient Permission', 'domain': 'global', 'reason': 'insufficientPermissions'}], 'status': 'PERMISSION_DENIED', 'details': [{'@type': 'type.googleapis.com/google.rpc.ErrorInfo', 'reason': 'ACCESS_TOKEN_SCOPE_INSUFFICIENT', 'domain': 'googleapis.com', 'metadata': {'method': 'google.apps.drive.v3.DriveFiles.List', 'service': 'drive.googleapis.com'}}]}

In [16]:
# Authorization
scope = ['https://www.googleapis.com/auth/spreadsheets']
creds = ServiceAccountCredentials.from_json_keyfile_name(ga_keys, scope)
service = build('sheets', 'v4', credentials=creds)

# Clear sheet data
spreadsheet_id = '1vUBjD3qVSZkNIgax4KbRVZ3u2aPiV533DawI0edZ7mQ'
sheet_range = 'WB_Товары_для_взрослых_Секс_игр!A:AA'  # set the range of cells to clear
request_body = {}
response = service.spreadsheets().values().clear(
    spreadsheetId=spreadsheet_id,
    range=sheet_range,
    body=request_body
).execute()

print('Sheet cleared successfully!')


Sheet cleared successfully!


In [18]:
df =SexToys_filteres_df


values = [df.columns.values.tolist()]

values.extend(df.values.tolist())

# Authorization
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
my_mail = 'account@proninsgs.iam.gserviceaccount.com'



credentials = ServiceAccountCredentials.from_json_keyfile_name(ga_keys, scope)

service = discovery.build('sheets', 'v4', credentials=credentials)
request = service.spreadsheets().values().update(spreadsheetId="1vUBjD3qVSZkNIgax4KbRVZ3u2aPiV533DawI0edZ7mQ", 
                                                 range="WB_Товары_для_взрослых_Секс_игр!A:aa",
                                                 valueInputOption="USER_ENTERED", body={'values': values})
response = request.execute()

# TODO: Change code below to process the `response` dict:
print(datetime.today())
pprint(response)

2023-06-22 10:26:56.022180
{'spreadsheetId': '1vUBjD3qVSZkNIgax4KbRVZ3u2aPiV533DawI0edZ7mQ',
 'updatedCells': 40660,
 'updatedColumns': 19,
 'updatedRange': "'WB_Товары_для_взрослых_Секс_игр'!A1:S2140",
 'updatedRows': 2140}
